<a href="https://colab.research.google.com/github/jrakhshanda/Text-Mining/blob/main/roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers
!pip install tokenizers
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
#!export XLA_USE_BF16=1

In [4]:
import os
import string
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import *
from transformers import AdamW, get_linear_schedule_with_warmup
import tokenizers

In [5]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## Configuration

In [6]:
from tokenizers import ByteLevelBPETokenizer
class config:
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 32
    EPOCHS = 4
    PATH = '/content/drive/MyDrive/RoBERTa_files'
    TRAINING_FILE = pd.read_csv(PATH+'/train.csv')
    TEST_FILE =  pd.read_csv(PATH+'/test.csv')
    MAX_LEN = 192
    TOKENIZER = ByteLevelBPETokenizer(f"{PATH}/vocab.json",
                                      f"{PATH}/merges.txt",
                                      lowercase=True, add_prefix_space=True)

# Processing of Data

In [7]:
def process_data(text, selected_text, sentiment, tokenizer=config.TOKENIZER, max_len=config.MAX_LEN):

    text = " " + " ".join(str(text).lower().split(" "))
    selected_text = " " + " ".join(str(selected_text).lower().split(" "))

    len_selected = len(selected_text) - 1
    idx1 = idx2 = None
    for idx, letter in enumerate(selected_text):
        if (text[idx] == selected_text[1]) and (" " + text[idx: idx+len_selected] == selected_text):
            idx1 = idx
            idx2 = idx1 + len_selected - 1
            break
    
    # making character targets
    if idx1!=None and idx2!=None:
        char_targets = [0] * len(text)
        for i in range(idx1, idx2+1):
            char_targets[i] = 1
    else:
        char_targets = [1] * len(text)

    # encoding using pretrained tokenizer
    tok_text = tokenizer.encode(text)
    ids = tok_text.ids
    attention_mask = tok_text.attention_mask
    type_ids = tok_text.type_ids
    offsets = tok_text.offsets

    # getting indexes of tokens containing character in selected_text
    target_idx = []
    for i, (offset1, offset2) in enumerate(offsets):
        if sum(char_targets[offset1: offset2])>0:
            target_idx.append(i)

    # we just need the indexes of the start and end tokens as we are using 
    # nn. CrossEntropy as loss
    start_target = target_idx[0]
    end_target = target_idx[-1]

    # token ids of sentiment as present in our vocab hard coded here
    sentiment_ids = {
        'positive':1313,                    # tokenizer.encode('positive').ids
        'negative':2430,                    # tokenizer.encode('negative').ids
        'neutral':7974                     # tokenizer.encode('neutral').ids
    }

    # adding special tokens
    input_ids = [0] + [sentiment_ids[sentiment]] + [2] + [2] + ids + [2]
    type_ids = [0, 0, 0, 0] + [0] * (len(ids) + 1)
    attention_mask = [1] * len(type_ids)
    offsets = [(0, 0)] * 4 + offsets
    start_target += 4
    end_target += 4

    # padding
    padding_len = max_len - len(ids)
    if padding_len>0:
        input_ids = input_ids + [1] * padding_len
        attention_mask = attention_mask + [0] * padding_len
        type_ids = type_ids + [0] * padding_len
        offsets = offsets + [(0, 0)] * padding_len

    return {
        'ids': torch.tensor(input_ids,dtype=torch.long),
        'attention_mask': torch.tensor(attention_mask,dtype=torch.long),
        'token_type_ids':torch.tensor(type_ids,dtype=torch.long),
        'targets_start': torch.tensor(start_target,dtype=torch.long),
        'targets_end':  torch.tensor(end_target,dtype=torch.long),
        'offsets': torch.tensor(offsets,dtype=torch.long),
        'padding_len': padding_len,
        'text': text,
        'selected_text': selected_text,
        'sentiment': sentiment
    }

In [8]:
class TextDataset(Dataset):
    def __init__(self, text, sentiment, selected_text):
        self.text = text
        self.sentiment = sentiment
        self.selected_text = selected_text

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        # processing data
        data = process_data(
            self.text[item], 
            self.selected_text[item], 
            self.sentiment[item]
        )
        # returning tensors
        return data

In [9]:
#import pdb
#pdb.set_trace()
df = config.TRAINING_FILE.reset_index(drop=True)
if __name__== "__main__":
  dset = TextDataset(text = df.text.values,
                      selected_text =df.selected_text.values,sentiment = df.sentiment.values)
  print(dset[5])

{'ids': tensor([    0,  7974,     2,     2,  2054,   640,  1401,     4,   417, 15244,
          428,  7928,  4469,     4,   175,    73,  9426,   506,   111,   103,
        36778, 10242,  3923,    13,     5,   275,   910, 15574,  7900,    15,
         6872,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,

Now we’ll create an iterator for our dataset using the torch DataLoader class. This helps save on memory during training because, unlike a for loop, with an iterator the entire dataset does not need to be loaded into memory.

## Model Implementation

In [10]:
class TextModel(BertPreTrainedModel):
    def __init__(self,conf):
        super(TextModel, self).__init__(conf)

        self.roberta = RobertaModel.from_pretrained("roberta-base",config = conf)
        self.drop_out = nn.Dropout(0.2)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        # this is to initialize the weights of the matrix that would convert 
        # (batch_size, max_len, 2*768) to (batch_size, max_len, 1) with std=0.02 
    
    def forward(self, ids, attention_mask, token_type_ids):
        _, _, output = self.roberta(ids,
                                    attention_mask = attention_mask,
                                    token_type_ids=token_type_ids).to_tuple()
        # out dim = (12, batch_size, max_len, 768)
        # 12 denotes the 12 hidden layers of roberta

        output = torch.cat((output[-1], output[-2]), dim=-1)
        # output dim = (batch_size, max_len, 2*768)
        output = self.drop_out(output)
        logits = self.l0(output)
        # logits dim -> (batch_size, max_len, 2)

        start_logits, end_logits = logits.split(1, dim=-1)
        # start_logits and end_logits dim -> (batch_size, max_len, 1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        # start_logits and end_logits dim -> (batch_size, max_len)

        return start_logits, end_logits

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The RoBERTa model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [12]:
# loss function. Play around with it and see what works best
def loss_fn(output_start, output_end, targets_start, targets_end,device):
  loss = nn.CrossEntropyLoss().to(device)
  l1 = loss(output_start,targets_start)
  l2 = loss(output_end,targets_end)
  return l1 + l2

In [13]:
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
t0 = time.time()

In [14]:
def train_fn(data_loader, model, optimizer, device, scheduler):
  total_loss = 0
  model.train()

  for bi, batch in enumerate(data_loader):
    # getting data
    ids = batch['ids'].to(device, dtype=torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
    targets_start = batch['targets_start'].to(device, dtype=torch.long)
    targets_end = batch['targets_end'].to(device, dtype=torch.long)

    # zeroing gradients
    optimizer.zero_grad()
    # getting outputs
    output_start, output_end = model(ids,
                                     attention_mask = attention_mask,
                                     token_type_ids=token_type_ids)
    # calulating loss
    loss = loss_fn(output_start, output_end, targets_start, targets_end,device)
    total_loss += loss.item()
    # calculating gradients
    loss.backward()
    # updating model parameters
    optimizer.step()
    # stepping learning rate scheduler
    scheduler.step()
  
  avg_train_loss = total_loss / len(data_loader)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
  return avg_train_loss

In [15]:
# jaccard function as mentioned in evaluation section of the contest
def jaccard_metric(text,
                   selected_text,
                   sentiment,
                   offsets,
                   start_idx,
                   end_idx): 
  
  if end_idx < start_idx:
    end_idx = start_idx
    
  pred  = ""
  for idx in range(start_idx, end_idx + 1):
    pred += text[offsets[idx][0]: offsets[idx][1]]
    if (idx+1) < len(offsets) and offsets[idx][1] < offsets[idx+1][0]:
      pred += " "

    if len(text.split()) < 5:
      pred = text
    
    if sentiment=='neutral':
      pred = text

  a = set(selected_text.lower().split()) 
  b = set(pred.lower().split())
  c = a.intersection(b)
  jacc = float(len(c)) / (len(a) + len(b) - len(c))
  return jacc, pred

In [16]:
def evaluate_fn(data_loader, model, device, tokenizer=config.TOKENIZER):
  total_loss = 0
  predicted_text = []
  model.eval()

  for bi, batch in enumerate(data_loader):
    # getting data
    ids = batch['ids'].to(device, dtype=torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
    targets_start = batch['targets_start'].to(device, dtype=torch.long)
    targets_end = batch['targets_end'].to(device, dtype=torch.long)
    pad_len = batch['padding_len']
    selected_text = batch['selected_text']
    text = batch['text']
    sentiment = batch['sentiment']
    offsets = batch["offsets"]

    # getting output
    output_start, output_end = model(ids,
                                     attention_mask = attention_mask,
                                     token_type_ids=token_type_ids)

    output_start = torch.softmax(output_start,dim=1).cpu().detach().numpy()
    output_end = torch.softmax(output_end,dim=1).cpu().detach().numpy()

    jaccard_similarity = []
    for px, tweet in enumerate(text):
      jacc, pred = jaccard_metric(tweet,
                                  selected_text[px],
                                  sentiment = sentiment[px],
                                  offsets = offsets[px],
                                  start_idx=np.argmax(output_start[px,:]),
                                  end_idx = np.argmax(output_end[px,:]))
      predicted_text.append(pred)  
      jaccard_similarity.append(jacc)
  
  print("  Average jaccard similarity: {0:.2f}".format(np.mean(jaccard_similarity)))
  print("  validation took: {:}".format(format_time(time.time() - t0)))

  return np.mean(jaccard_similarity), predicted_text

In [17]:
# reading train.csv
dfx = config.TRAINING_FILE.head(500)
dfx.reset_index(drop=True)
# spliting into training and validation set
train, valid = model_selection.train_test_split(dfx,
                                                      test_size=0.1,
                                                      random_state=42,
                                                      stratify=dfx.sentiment.values)

# using TextDataset function as coded above
train_dataset = TextDataset(text=train.text.values,
                            sentiment=train.sentiment.values,
                            selected_text=train.selected_text.values)

valid_dataset = TextDataset(text=valid.text.values,
                            sentiment=valid.sentiment.values,
                            selected_text=valid.selected_text.values)

# making pytorch dataloaders
train_dataloader = DataLoader(train_dataset,
                              batch_size=config.TRAIN_BATCH_SIZE)

valid_dataloader = DataLoader(valid_dataset,
                              batch_size=config.VALID_BATCH_SIZE)

conf = RobertaConfig.from_pretrained(f"{config.PATH}/config.json")
conf.output_hidden_states = True
model = TextModel(conf)
model.to(device)
    
# explicitly going through model parameters and removing weight decay
# from a few layers 
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay':1e-5},
                        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0},
                        ]

# Coding out the optimizer and scheduler

num_train_steps = int(len(train_dataloader) * config.EPOCHS)

optimizer = AdamW(optimizer_parameters, lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_train_steps)

In [ ]:
import random

# Store the average loss after each epoch so we can plot them.

train_loss = []
jaccards = []
best_jaccard = 0

for epoch in range(0, config.EPOCHS):
  # ========================================
  #               Training
  # ========================================
    
  # Perform one full pass over the training set.
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch+1, config.EPOCHS))
  print('Training...')

  # Measure how long the training epoch takes.
  t0 = time.time()
  
  avg_train_loss = train_fn(train_dataloader,model,optimizer,device,scheduler)
  train_loss.append(avg_train_loss)
  
  # ========================================
  #               Validation
  # ========================================
  print("")
  print("Running Validation...")
  t0 = time.time()
  jaccard, predictions = evaluate_fn(valid_dataloader, model, device)
  jaccards.append(jaccard)

  if jaccard > best_jaccard:
    print('saving model')
    torch.save(model.state_dict(), config.PATH+'/roberta_modeljcc.pth')
    best_jaccard = jaccard

  if epoch == 4:
    print('saving check_point')
    torch.save(model.state_dict(), config.PATH+"/roberta_model3.pth")



======== Epoch 1 / 4 ========
Training...


In [ ]:
conf = RobertaConfig.from_pretrained(f"{config.PATH}/config.json")
conf.output_hidden_states = True
model = TextModel(conf)
model.to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/RoBERTa_files/roberta_model1.pth"))

In [ ]:
data = {'jaccard':jaccards, 
        'train_loss':train_loss} 
  
losses = pd.DataFrame(data) 

In [ ]:
from google.colab import files
df.to_csv('losses.csv') 
files.download('losses.csv')

In [ ]:
total_loss = 0
model.eval()

for bi, batch in enumerate(data_loader):
  # getting data
  ids = batch['ids'].to(device, dtype=torch.long)
  token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
  attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
  targets_start = batch['targets_start'].to(device, dtype=torch.long)
  targets_end = batch['targets_end'].to(device, dtype=torch.long)
  selected_text = batch['selected_text']
  pad_len = batch['padding_len']
  text = batch['text']
  sentiment = batch['sentiment']
  offsets = batch['offsets']
  # getting output
  output_start, output_end = model(ids,attention_mask = attention_mask,token_type_ids=token_type_ids)
    
    
  loss = loss_fn(output_start, output_end, targets_start, targets_end,device)
  total_loss += loss.item()

  output_start = torch.softmax(output_start, dim=1).cpu().detach().numpy()
  output_end = torch.softmax(output_end, dim=1).cpu().detach().numpy()

  jaccard_similarity = []
  predicted_text = []
  for px, tweet in enumerate(text):
    sel_text = selected_text[px]
    start_idx = np.argmax(output_start[px,:])
    end_idx = np.argmax(output_end[px,:])
    offs = offsets[px]
    
    if end_idx < start_idx:
      end_idx = start_idx
    
    pred  = ""
    for idx in range(start_idx, end_idx + 1):
      pred += tweet[offs[idx][0]: offs[idx][1]]
      if (idx+1) < len(offs) and offs[idx][1] < offs[idx+1][0]:
        pred += " "

    if len(tweet.split()) < 2:
      pred = tweet
    jacc =jaccard_metric(sel_text,pred)

    predicted_text.append(pred)  
    jaccard_similarity.append(jacc)

In [ ]:
jaccs, predictions = evaluate_fn(train_dataloader,model,device,config.TOKENIZER)

  Average validation loss: 232.30
  Average jaccard similarity: 0.73
  validation took: 0:06:41


In [ ]:
jaccs

0.726385717457146